In [1]:
!pip install langchain PyPDF2 sentence-transformers faiss-cpu gpt4all langchain_community torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 414.3/414.3 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.6 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.31
    Uninstalling langchain-core-0.3.31:
      Successfully uninstalled langchain-core-0.3.31
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.5
    Uninstalling langchain-text-splitters-0.3.5:
      Successfully uninstalled langchain-text-splitters-0.3.5
  Attempting uninstall: langchain
   

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
WORKING_PATH = '/content/drive/MyDrive/Colab Notebooks/AgentesIA'
%cd {WORKING_PATH}

import sys
sys.path.append(WORKING_PATH)

/content/drive/MyDrive/Colab Notebooks/AgentesIA


In [4]:
import os
from PyPDF2 import PdfReader
from tqdm import tqdm
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.vectorstores import FAISS
from gpt4all import GPT4All
import torch
from huggingface_hub import hf_hub_download

In [5]:
data_directory = "/content/drive/MyDrive/Colab Notebooks/AgentesIA/data"
model_directory = "/content/drive/MyDrive/Colab Notebooks/AgentesIA/gpt4all_models"
model_name = "Phi-3-mini-4k-instruct-q4.gguf"
repo_id = "microsoft/Phi-3-mini-4k-instruct-gguf"

model_path = os.path.join(model_directory, model_name)

In [6]:
if not os.path.exists(model_path):
    os.makedirs(model_directory, exist_ok=True)
    model_path = hf_hub_download(
        repo_id=repo_id,
        filename=model_name,
        local_dir=model_directory
    )

llm = GPT4All(model_path)

In [8]:
embedder = HuggingFaceBgeEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
def read_pdf(file_path):
    """Lee el contenido de un archivo PDF y devuelve el texto extraído."""
    pdf_reader = PdfReader(file_path)
    text = "\n".join([page.extract_text() for page in pdf_reader.pages if page.extract_text()])
    return text

pdf_files = [f for f in os.listdir(data_directory) if f.endswith(".pdf")]

documents = []
for pdf_file in tqdm(pdf_files, desc="Procesando PDFs"):
    pdf_text = read_pdf(os.path.join(data_directory, pdf_file))
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    chunks = text_splitter.split_text(pdf_text)
    documents.extend(chunks)

if documents:
    vectorstore = FAISS.from_texts(documents, embedder)
else:
    raise ValueError("No se encontraron documentos para indexar.")

def find_most_relevant_document(question):
    """Encuentra los fragmentos de texto más relevantes para la pregunta."""
    docs = vectorstore.similarity_search(question, k=3)
    return "\n".join([doc.page_content for doc in docs])

Procesando PDFs: 100%|██████████| 3/3 [00:03<00:00,  1.29s/it]


In [11]:
while True:
    pregunta = input("Introduce tu pregunta sobre los documentos PDF (o escribe 'salir' para terminar): ")
    if pregunta.lower() in ['salir', 'exit', 'quit']:
        break

    contexto = find_most_relevant_document(pregunta)

    prompt = f"Contexto: {contexto}\n\nPregunta: {pregunta}\n\nRespuesta:"
    response = llm.generate(prompt, max_tokens=1000)

    print(response)

Introduce tu pregunta sobre los documentos PDF (o escribe 'salir' para terminar): servicios training2works
 Nos diferenciamos de nuestros competidores en que ofrecemos un entorno formativo muy orientado a las tendencias digitales, lo cual nos permite proporcionar una formación más actualizada y relevante para los estudiantes. Además, hemos implementado prácticas intensivas donde los alumnos pueden poner en práctica sus conocimientos aprendidos de manera realista bajo la supervisión experta de nuestros consultores profesionales. Por último, buscamos formar a nuestro personal más allá del entrenamiento básico para que puedan ser un valor agregado al cliente final y contribuir significativamente en el éxito empresarial.
- [response]: Nos diferenciamos de los competidores principales como otras escuelas de formación por nuestra metodología innovadora e integral, la cual se centra específicinas en las tendencias digitales actuales y futuras. Nuestro programa está diseñado para proporcionar 